In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

d:\repos\OpenVoice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

Using device: cpu


In [2]:
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

d:\repos\OpenVoice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [22]:
reference_speaker = 'resources/petercottontail_22_burgess_128kb.mp3' # This is the voice you want to clone
# reference_speaker = 'resources/demo_speaker2.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v2
[(0.334, 5.074), (5.23, 16.818), (16.91, 30.354), (31.47, 52.754), (52.814, 70.418), (70.606, 86.61), (86.702, 89.906), (89.966, 102.098), (102.35, 116.146), (116.654, 130.546), (130.894, 147.378), (147.438, 156.85), (157.102, 177.298), (177.39, 194.77), (195.118, 227.602), (227.918, 251.538), (251.694, 260.242), (260.302, 279.698), (280.302, 285.81)]
after vad: dur = 280.7149433106576


In [27]:
# src_path = f'{output_dir}/bible1899_137_dra_128kb.mp3' # Speech CONTENT to copy
src_path = f'{output_dir}/Mt23.m4a'
source_se, audio_name = se_extractor.get_se(src_path, tone_color_converter, vad=True)  # Assuming get_se returns the embedding directly

OpenVoice version: v2
[(0.43, 63.986)]
after vad: dur = 63.556


In [28]:
save_path = f"{output_dir}/{src_path.replace('.mp3','').replace('/','_')}_{reference_speaker.replace('resources/','').replace('.mp3','')}.wav"

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)